In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/HGFN.db")

tables = ["inventory", "sku_master", "orders", "suppliers", "warehouse"]
views = ["orders_clean", "inventory_clean", "sku_clean", "supplier_clean", "warehouse_clean"]
for t in tables: 
    count = pd.read_sql(f"SELECT COUNT (*) AS rows FROM {t};", conn)
    print(t)
    display(count)
for v in views: 
    count = pd.read_sql(f"SELECT COUNT (*) AS rows FROM {v};", conn)
    print(v)
    display(count)

inventory


,rows
0,120000


sku_master


,rows
0,10000


orders


...

,rows
0,10000


supplier_clean


,rows
0,30000


warehouse_clean


,rows
0,5480


In [15]:
scc = """ 
SELECT * 
FROM supplier_clean
LIMIT 5
"""
supplier_clean_check = pd.read_sql(scc, conn)
supplier_clean_check 

,supplier_id,shipment_id,order_date,promised_lead_time,actual_lead_time,units_ordered,units_received,delivery_status,cost_per_unit,fill_rate,lead_variance
0,SUP011,SHP000001,2022,8,12,69,69,Late,68.30,1.0,4
1,SUP026,SHP000002,2022,20,23,330,330,Late,179.33,1.0,3
2,SUP038,SHP000003,2023,20,23,464,464,Late,135.43,1.0,3
3,SUP019,SHP000004,2023,11,9,394,394,Early,88.34,1.0,-2
4,SUP011,SHP000005,2023,19,22,476,476,Late,163.90,1.0,3


In [26]:
otr = """ 
SELECT supplier_id, 
    AVG(actual_lead_time < promised_lead_time) * 1.0 AS on_time_rate,
       actual_lead_time, 
       promised_lead_time
FROM supplier_clean
GROUP BY supplier_id
ORDER BY on_time_rate DESC
LIMIT 5
"""
on_time_rate = pd.read_sql(otr, conn)
on_time_rate

,supplier_id,on_time_rate,actual_lead_time,promised_lead_time
0,SUP039,0.552182,19,20
1,SUP015,0.550000,16,18
2,SUP002,0.549407,21,19
3,SUP040,0.545842,13,17
4,SUP060,0.542268,1,3


In [27]:
occ = """ 
SELECT * 
FROM orders_clean
LIMIT 5
"""
orders_clean_check = pd.read_sql(occ, conn)
orders_clean_check

,order_id,sku_id,warehouse_id,order_date,fulfillment_date,region,shipping_method,quantity,fulfillment_time_hours,delivery_time_days,days_to_fulfill
0,ORD0000001,SKU05505,MFC,2024,2024,Midwest,Standard,3,45.72,6,1.905032
1,ORD0000002,SKU02542,CFC,2024,2024,Central,Standard,3,41.35,5,1.722992
2,ORD0000003,SKU05924,CFC,2023,2023,Central,Standard,2,52.81,4,2.200275
3,ORD0000004,SKU05295,MFC,2024,2024,Midwest,Standard,1,33.88,5,1.411482
4,ORD0000005,SKU00424,SFC,2024,2024,South,Standard,1,25.71,1,1.071309


In [48]:
smp = """ 
SELECT 
    shipping_method, 
    delivery_time_days, 
    days_to_fulfill
FROM 
    orders_clean
GROUP BY 
    shipping_method
ORDER BY 
    delivery_time_days ASC
"""
shipping_method_performance = pd.read_sql(smp, conn)
shipping_method_performance

,shipping_method,delivery_time_days,days_to_fulfill
0,Next-Day,1,0.619424
1,Expedited,4,0.422019
2,Standard,6,1.905032


In [50]:
icc = """ 
SELECT *
FROM inventory_clean 
LIMIT 5 
"""
inventory_clean_check = pd.read_sql(icc, conn)
inventory_clean_check

,movement_id,sku_id,warehouse_id,movement_type,quantity,movement_date,starting_inventory_level,ending_inventory_level,reorder_point,safety_stock,inventory_change
0,MOV0000001,SKU09198,WFC,Inbound,291,2024,659,950,154,124,291
1,MOV0000002,SKU00634,SFC,Outbound,49,2022,59,10,332,100,-49
2,MOV0000003,SKU03516,CFC,Outbound,133,2022,309,176,337,185,-133
3,MOV0000004,SKU00385,WFC,Outbound,19,2023,895,876,292,102,-19
4,MOV0000005,SKU04952,CFC,Outbound,119,2023,547,428,182,72,-119


In [54]:
im = """ 
SELECT 
    sku_id,
    movement_type, 
    SUM(starting_inventory_level - ending_inventory_level) AS total_sku_movement
FROM 
    inventory_clean
GROUP BY 
    sku_id, movement_type
ORDER BY 
    total_sku_movement DESC
LIMIT 5
    """
inventory_movement = pd.read_sql(im, conn)
inventory_movement

,sku_id,movement_type,total_sku_movement
0,SKU05809,Outbound,1726
1,SKU00654,Outbound,1707
2,SKU03666,Outbound,1698
3,SKU05645,Outbound,1597
4,SKU04665,Outbound,1560


In [68]:
wcc = """ 
SELECT * 
FROM warehouse_clean
LIMIT 5
"""

warehouse_clean_check = pd.read_sql(wcc, conn)
warehouse_clean_check

,warehouse_id,work_date,picks_per_hour,lines_per_hour,receiving_volume,outbound_volume,labor_hours,errors,rework,units_per_labor_hour,error_rate
0,EFC,2022,83.08,66.78,865,459,518.37,1,0,0.885468,0.002179
1,EFC,2022,77.85,63.95,674,843,571.57,4,2,1.474885,0.004745
2,EFC,2022,103.24,92.07,574,987,418.25,2,0,2.359833,0.002026
3,EFC,2022,67.96,64.23,1029,952,722.69,0,0,1.317301,0.000000
4,EFC,2022,52.57,51.67,836,1012,406.15,2,1,2.491690,0.001976


In [83]:
wp = """ 
SELECT 
    warehouse_id, 
    SUM(picks_per_hour + lines_per_hour) AS total_operations_per_hour,
    units_per_labor_hour, 
    errors
FROM 
    warehouse_clean
GROUP BY 
    warehouse_id
ORDER BY 
    units_per_labor_hour DESC
LIMIT 5
"""
warehouse_performance = pd.read_sql(wp, conn)
warehouse_performance


,warehouse_id,total_operations_per_hour,units_per_labor_hour,errors
0,SFC,173389.82,2.535211,2
1,WFC,177074.17,1.605269,4
2,MFC,175429.97,1.497042,3
3,CFC,176088.43,1.348875,5
4,EFC,176115.46,0.885468,1


In [77]:
scc = """ 
SELECT * 
FROM sku_clean
LIMIT 5"""

sku_clean_check = pd.read_sql(scc, conn)
sku_clean_check 

,sku_id,sku_name,category,sub_category,cost,price,supplier_id,weight,profit_per_unit,margin_pct
0,SKU00001,Product SKU00001,Home & Kitchen,Appliances,76.16,185.52,SUP048,36.63,109.36,1.435924
1,SKU00002,Product SKU00002,Health & Beauty,Skincare,120.53,169.08,SUP015,7.88,48.55,0.402804
2,SKU00003,Product SKU00003,Apparel,Kids,13.50,31.40,SUP007,30.10,17.90,1.325926
3,SKU00004,Product SKU00004,Grocery,Dry Goods,142.20,174.45,SUP038,48.50,32.25,0.226793
4,SKU00005,Product SKU00005,Automotive,Parts,166.82,246.23,SUP002,9.17,79.41,0.476022


In [79]:
ps = """ 
SELECT 
    sku_id, 
    category, 
    profit_per_unit,
    margin_pct
FROM 
    sku_clean
GROUP BY 
    category
ORDER BY 
    profit_per_unit DESC
LIMIT 5
"""

profit_sku = pd.read_sql(ps, conn)
profit_sku

,sku_id,category,profit_per_unit,margin_pct
0,SKU00001,Home & Kitchen,109.36,1.435924
1,SKU00010,Office Supplies,100.72,0.970141
2,SKU00005,Automotive,79.41,0.476022
3,SKU00011,Electronics,51.57,0.421703
4,SKU00002,Health & Beauty,48.55,0.402804
